# **PySpark**: Apache Spark Python API

## 1. Spark Cluster

### 1.1. Connection

Для использования Spark необходимо создать SparkSession с параметрами:

+ **appName:** имя в [Spark Master Web UI](http://localhost:8080/);
+ **master:** Spark Master URL, то что использует Spark Workers;
+ **spark.executor.memory:** кол-во ОЗУ для executor'a, должно быть не больше чем SPARK_WORKER_MEMORY в конфиге.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

SparkSession имеет и больше конфигураций, их можно указать в **config** методах. Все описания есть в [документации](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SparkSession).

## 2. Data

### 2.1. Read

Прочитаем данные из директории ([данные с Kaggle](https://www.kaggle.com/bank-of-england/a-millennium-of-macroeconomic-data)) 

In [2]:
data = spark.read.csv(path="data/uk-macroeconomic-data.csv", sep=",", header=True)

In [3]:
# кол-во строк
data.count()

841

In [4]:
# кол-во колонок
len(data.columns)

77

In [ ]:
# вывод структуры таблицы на монитор
data.printSchema()

### 2.2. Process

Пример работы с данными

In [6]:
unemployment = data.select(["Description", "Population (GB+NI)", "Unemployment rate"])

In [7]:
# показать первые 10 строк 
unemployment.show(n=10)

+-----------+------------------+-----------------+
|Description|Population (GB+NI)|Unemployment rate|
+-----------+------------------+-----------------+
|      Units|              000s|                %|
|       1209|              null|             null|
|       1210|              null|             null|
|       1211|              null|             null|
|       1212|              null|             null|
|       1213|              null|             null|
|       1214|              null|             null|
|       1215|              null|             null|
|       1216|              null|             null|
|       1217|              null|             null|
+-----------+------------------+-----------------+
only showing top 10 rows



In [8]:
# фильтр по условию
cols_description = unemployment.filter(unemployment['Description'] == 'Units')

In [9]:
cols_description.show()

+-----------+------------------+-----------------+
|Description|Population (GB+NI)|Unemployment rate|
+-----------+------------------+-----------------+
|      Units|              000s|                %|
+-----------+------------------+-----------------+



In [10]:
# пример соединения
unemployment = unemployment.join(other=cols_description, on=['Description'], how='left_anti')

In [11]:
unemployment.show(n=10)

+-----------+------------------+-----------------+
|Description|Population (GB+NI)|Unemployment rate|
+-----------+------------------+-----------------+
|       1209|              null|             null|
|       1210|              null|             null|
|       1211|              null|             null|
|       1212|              null|             null|
|       1213|              null|             null|
|       1214|              null|             null|
|       1215|              null|             null|
|       1216|              null|             null|
|       1217|              null|             null|
|       1218|              null|             null|
+-----------+------------------+-----------------+
only showing top 10 rows



In [12]:
# удаление пустот
unemployment = unemployment.dropna()

In [13]:
# добавление новых колонок
unemployment = unemployment.\
               withColumnRenamed("Description", 'year').\
               withColumnRenamed("Population (GB+NI)", "population").\
               withColumnRenamed("Unemployment rate", "unemployment_rate")

In [14]:
unemployment.show(n=10)

+----+----------+-----------------+
|year|population|unemployment_rate|
+----+----------+-----------------+
|1855|     23241|             3.73|
|1856|     23466|             3.52|
|1857|     23689|             3.95|
|1858|     23914|             5.23|
|1859|     24138|             3.27|
|1860|     24360|             2.94|
|1861|     24585|             3.72|
|1862|     24862|             4.68|
|1863|     25142|             4.15|
|1864|     25425|             2.99|
+----+----------+-----------------+
only showing top 10 rows



### 2.3. Write

In [15]:
# запись на диск
unemployment.\
    repartition(1).\
    write.\
    csv(path="data/uk-macroeconomic-unemployment-data.csv", sep=",", header=True, mode="overwrite")